In [ ]:
pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━

In [ ]:
import pandas as pd
df_train = pd.read_csv("/kaggle/input/superai5-stock-price-prediction/X_train.csv")
df_test = pd.read_csv("/kaggle/input/superai5-stock-price-prediction/X_test.csv")
y_train = pd.read_csv("/kaggle/input/superai5-stock-price-prediction/y_train.csv")

In [ ]:
df_train['Close'] = y_train['Close']

In [ ]:
df_train.columns

Index(['Id', 'High', 'Low', 'Open', 'Volume', 'Close'], dtype='object')

In [ ]:
df_train = df_train.drop(["Id",'Volume'],axis=1)
df_test = df_test.drop(["Id",'Volume'],axis=1)
df_train.head(3)

,High,Low,Close
0,1528.699951,1518.000000,1524.500000
1,1552.699951,1530.099976,1549.199951
2,1580.000000,1560.400024,1566.199951


# model

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
hyperparameters = {
    'GBM': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'CAT': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'XGB': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
    'RF': [
        {'ag_args_fit': {'num_gpus': 0}},  # Train with CPU
        {'ag_args_fit': {'num_gpus': 1}}   # Train with GPU
    ],
}
time_limit=180

In [ ]:
predictor = TabularPredictor(label='Close',
                            problem_type='regression',
                            # path=save_path,
                            eval_metric='mean_squared_error'
                            ).fit(
                                df_train,
                                presets='best_quality',
                                hyperparameters=hyperparameters,
                                time_limit=time_limit
                            )
print("Finish!")

No path specified. Models will be saved in: "AutogluonModels/ag-20250228_182016"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.14 GB / 31.35 GB (92.9%)
Disk Space Avail:   18.96 GB / 19.52 GB (97.1%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

Finish!


In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-90.008555,mean_squared_error,0.181083,35.155502,0.000660,0.044672,2,True,9
1,CatBoost_BAG_L1,-91.576263,mean_squared_error,0.013633,8.923370,0.013633,8.923370,1,True,5
2,CatBoost_2_BAG_L1,-100.745133,mean_squared_error,0.010608,26.090679,0.010608,26.090679,1,True,6
3,RandomForest_2_BAG_L1,-102.678992,mean_squared_error,0.104759,0.539753,0.104759,0.539753,1,True,4
4,RandomForest_BAG_L1,-102.678992,mean_squared_error,0.117399,0.631086,0.117399,0.631086,1,True,3
5,XGBoost_2_BAG_L1,-107.874683,mean_squared_error,0.038813,11.414118,0.038813,11.414118,1,True,8
6,XGBoost_BAG_L1,-107.874756,mean_squared_error,0.029805,5.778550,0.029805,5.778550,1,True,7
7,LightGBM_2_BAG_L1,-138.864825,mean_squared_error,0.010579,14.142255,0.010579,14.142255,1,True,2
8,LightGBM_BAG_L1,-138.868395,mean_squared_error,0.013909,9.341592,0.013909,9.341592,1,True,1


In [ ]:
# predictor.feature_importance(df_train)

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/superai5-stock-price-prediction/sample_submission.csv")

In [ ]:
y_pred = predictor.predict(df_test)
y_pred

0      1725.050903
1      1724.779785
2      1717.346436
3      1718.747681
4      1743.971680
          ...     
638    2031.540161
639    2031.540161
640    2031.540161
641    2031.540161
642    2031.540161
Name: Close, Length: 643, dtype: float32

In [ ]:
sample_submission['Close'] = y_pred
sample_submission

,Id,Close
0,643,1725.050903
1,644,1724.779785
2,645,1717.346436
3,646,1718.747681
4,647,1743.971680
...,...,...
638,1281,2031.540161
639,1282,2031.540161
640,1283,2031.540161
641,1284,2031.540161


In [ ]:
sample_submission.to_csv("MSE_Stock_180sec_4model_drop_Id_Volumn_open_no_batch.csv",index=False)